In [1]:
import sqlite3

conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [2]:
cur.execute('''SELECT DISTINCT from_id FROM Links''')
from_ids = list()
for row in cur: 
    from_ids.append(row[0])
to_ids = list()
links = list()
cur.execute('''SELECT DISTINCT from_id, to_id FROM Links''')
for row in cur:
    from_id = row[0]
    to_id = row[1]
    if from_id == to_id : continue
    if from_id not in from_ids : continue
    if to_id not in from_ids : continue
    links.append(row)
    if to_id not in to_ids : to_ids.append(to_id)

In [3]:
prev_rank = dict()
for node in from_ids:
    cur.execute('''SELECT new_rank FROM Pages WHERE id = ?''', (node, ))
    row = cur.fetchone()
    prev_rank[node] = row[0]
sval = raw_input('How many iterations:')
num = 1
if ( len(sval) > 0 ) : num = int(sval)
if len(prev_rank) < 1 : 
    print "Nothing to rank."
    quit()

How many iterations:20


In [4]:
for i in range(num):
    next_ranks = dict();
    total = 0.0
    for (node, old_rank) in prev_rank.items():
        total += old_rank
        next_ranks[node] = 0.0
    for (node, old_rank) in prev_rank.items():
        give_ids = list()
        for (from_id, to_id) in links:
            if from_id != node : continue
            if to_id not in to_ids: continue
            give_ids.append(to_id)
        if ( len(give_ids) < 1 ) : continue
        amount = old_rank / len(give_ids)
    
        for id in give_ids:
            next_ranks[id] += amount
    
    newtot = 0
    for (node, next_rank) in next_ranks.items():
        newtot += next_rank
    factor = (total - newtot) / len(next_ranks)

    for node in next_ranks:
        next_ranks[node] += factor

    newtot = 0
    for (node, next_rank) in next_ranks.items():
        newtot += next_rank
    totdiff = 0
    for (node, old_rank) in prev_rank.items():
        new_rank = next_ranks[node]
        diff = abs(old_rank-new_rank)
        totdiff += diff

    avediff = totdiff / len(prev_rank)
    prev_rank = next_ranks

In [5]:
cur.execute('''UPDATE Pages SET old_rank=new_rank''')
for (id, new_rank) in next_ranks.items() :
    cur.execute('''UPDATE Pages SET new_rank=? WHERE id=?''', (new_rank, id))
conn.commit()
cur.close()